# Company Bankruptcy Prediction

## By Minh Nguyet Nguyen (Selene)

This project aims to predict company bankruptcy by applying machine learning techniques, specifically Support Vector Machines (SVM) and Random Forest classifiers. The project explores different design choices, compares model performance, and analyzes time-space complexity to determine the most effective approach for this classification task.

In [1]:
# Python Version 3.9.13, conda version 23.1.0

In [7]:
# Import packages
import sklearn # v 1.0.2
import numpy as np # v 1.21.5
import pandas as pd # v 1.4.4
import matplotlib.pyplot as plt # v 3.5.2
import statsmodels.api as sm # v 0.13.2
import sweetviz as sv # v 2.2.1
import statsmodels.tsa.stattools as stattools # v 0.13.2
import warnings as ww

from sklearn.model_selection import train_test_split # v 1.0.2
from sklearn.ensemble import RandomForestClassifier # v 1.0.2


from sklearn.tree import DecisionTreeRegressor # v 1.0.2
from sklearn.svm import SVC # v 1.0.2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix # v 1.0.2
from sklearn.tree import export_text # v 1.0.2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS # v 0.23.0
from sklearn.utils import resample # v 1.0.2

from sklearn.impute import SimpleImputer # v 1.0.2 # Impute missing values

import seaborn as sns # v. 0.11.2

ww.filterwarnings("ignore")
%matplotlib inline

In [8]:
# Read in the dataset
bankruptcy_df = pd.read_csv("/content/company_bankruptcy_data.csv")

## EDA

In [9]:
bankruptcy_df.columns

Index(['Bankrupt', ' ROA(C) before interest and depreciation before interest',
       ' ROA(A) before interest and % after tax',
       ' ROA(B) before interest and depreciation after tax',
       ' Operating Gross Margin', ' Realized Sales Gross Margin',
       ' Operating Profit Rate', ' Pre-tax net Interest Rate',
       ' After-tax net Interest Rate',
       ' Non-industry income and expenditure/revenue',
       ' Continuous interest rate (after tax)', ' Operating Expense Rate',
       ' Research and development expense rate', ' Cash flow rate',
       ' Interest-bearing debt interest rate', ' Tax rate (A)',
       ' Net Value Per Share (B)', ' Net Value Per Share (A)',
       ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons',
       ' Cash Flow Per Share', ' Revenue Per Share (Yuan ¥)',
       ' Operating Profit Per Share (Yuan ¥)',
       ' Per Share Net profit before tax (Yuan ¥)',
       ' Realized Sales Gross Profit Growth Rate',
       ' Operating Profit 

In [10]:
bankruptcy_df.shape

(6819, 96)

In [11]:
bankruptcy_df.dtypes

Bankrupt                                                      int64
 ROA(C) before interest and depreciation before interest    float64
 ROA(A) before interest and % after tax                     float64
 ROA(B) before interest and depreciation after tax          float64
 Operating Gross Margin                                     float64
                                                             ...   
 Liability to Equity                                        float64
 Degree of Financial Leverage (DFL)                         float64
 Interest Coverage Ratio (Interest expense to EBIT)         float64
 Net Income Flag                                              int64
 Equity to Liability                                        float64
Length: 96, dtype: object

In [12]:
# Missing values
bankruptcy_df.isna().sum() # the total number of missing values (NaN or null) in each column of the dataframe

Bankrupt                                                    0
 ROA(C) before interest and depreciation before interest    0
 ROA(A) before interest and % after tax                     0
 ROA(B) before interest and depreciation after tax          0
 Operating Gross Margin                                     0
                                                           ..
 Liability to Equity                                        0
 Degree of Financial Leverage (DFL)                         0
 Interest Coverage Ratio (Interest expense to EBIT)         0
 Net Income Flag                                            0
 Equity to Liability                                        0
Length: 96, dtype: int64

In [13]:
# Find columns with maximum value greater than 1
columns_with_max_greater_than_1 = bankruptcy_df.columns[bankruptcy_df.max() > 1]
columns_with_max_greater_than_1

Index([' Operating Expense Rate', ' Research and development expense rate',
       ' Interest-bearing debt interest rate', ' Revenue Per Share (Yuan ¥)',
       ' Total Asset Growth Rate', ' Net Value Growth Rate', ' Current Ratio',
       ' Quick Ratio', ' Total debt/Total net worth',
       ' Accounts Receivable Turnover', ' Average Collection Days',
       ' Inventory Turnover Rate (times)', ' Fixed Assets Turnover Frequency',
       ' Revenue per person', ' Allocation rate per person',
       ' Quick Assets/Current Liability', ' Cash/Current Liability',
       ' Inventory/Current Liability',
       ' Long-term Liability to Current Assets',
       ' Current Asset Turnover Rate', ' Quick Asset Turnover Rate',
       ' Cash Turnover Rate', ' Fixed Assets to Assets',
       ' Total assets to GNP price'],
      dtype='object')

In [14]:
# First 5 rows of the dataset
bankruptcy_df.head()

Bankrupt   ROA(C) before interest and depreciation before interest  \
0         1                                           0.370594          
1         1                                           0.464291          
2         1                                           0.426071          
3         1                                           0.399844          
4         1                                           0.465022          

    ROA(A) before interest and % after tax  \
0                                 0.424389   
1                                 0.538214   
2                                 0.499019   
3                                 0.451265   
4                                 0.538432   

    ROA(B) before interest and depreciation after tax  \
0                                           0.405750    
1                                           0.516730    
2                                           0.472295    
3                                           0.457733    
4                                           0.522298    

    Operating Gross Margin   Realized Sales Gross Margin  \
0                 0.601457                      0.601457   
1                 0.610235                      0.610235   
2                 0.601450                      0.601364   
3                 0.583541                      0.583541   
4                 0.598783                      0.598783   

    Operating Profit Rate   Pre-tax net Interest Rate  \
0                0.998969                    0.796887   
1                0.998946                    0.797380   
2                0.998857                    0.796403   
3                0.998700                    0.796967   
4                0.998973                    0.797366   

    After-tax net Interest Rate   Non-industry income and expenditure/revenue  \
0                      0.808809                                      0.302646   
1                      0.809301                                      0.303556   
2                      0.808388                                      0.302035   
3                      0.808966                                      0.303350   
4                      0.809304                                      0.303475   

   ...   Net Income to Total Assets   Total assets to GNP price  \
0  ...                     0.716845                    0.009219   
1  ...                     0.795297                    0.008323   
2  ...                     0.774670                    0.040003   
3  ...                     0.739555                    0.003252   
4  ...                     0.795016                    0.003878   

    No-credit Interval   Gross Profit to Sales  \
0             0.622879                0.601453   
1             0.623652                0.610237   
2             0.623841                0.601449   
3             0.622929                0.583538   
4             0.623521                0.598782   

    Net Income to Stockholder's Equity   Liability to Equity  \
0                             0.827890              0.290202   
1                             0.839969              0.283846   
2                             0.836774              0.290189   
3                             0.834697              0.281721   
4                             0.839973              0.278514   

    Degree of Financial Leverage (DFL)  \
0                             0.026601   
1                             0.264577   
2                             0.026555   
3                             0.026697   
4                             0.024752   

    Interest Coverage Ratio (Interest expense to EBIT)   Net Income Flag  \
0                                           0.564050                   1   
1                                           0.570175                   1   
2                                           0.563706                   1   
3                                           0.564663                   1   
4                              

In [15]:
# Last 5 rows of the dataset
bankruptcy_df.tail()

Bankrupt   ROA(C) before interest and depreciation before interest  \
6814         0                                           0.493687          
6815         0                                           0.475162          
6816         0                                           0.472725          
6817         0                                           0.506264          
6818         0                                           0.493053          

       ROA(A) before interest and % after tax  \
6814                                 0.539468   
6815                                 0.538269   
6816                                 0.533744   
6817                                 0.559911   
6818                                 0.570105   

       ROA(B) before interest and depreciation after tax  \
6814                                           0.543230    
6815                                           0.524172    
6816                                           0.520638    
6817                                           0.554045    
6818                                           0.549548    

       Operating Gross Margin   Realized Sales Gross Margin  \
6814                 0.604455                      0.604462   
6815                 0.598308                      0.598308   
6816                 0.610444                      0.610213   
6817                 0.607850                      0.607850   
6818                 0.627409                      0.627409   

       Operating Profit Rate   Pre-tax net Interest Rate  \
6814                0.998992                    0.797409   
6815                0.998992                    0.797414   
6816                0.998984                    0.797401   
6817                0.999074                    0.797500   
6818                0.998080                    0.801987   

       After-tax net Interest Rate  \
6814                      0.809331   
6815                      0.809327   
6816                      0.809317   
6817                      0.809399   
6818                      0.813800   

       Non-industry income and expenditure/revenue  ...  \
6814                                      0.303510  ...   
6815                                      0.303520  ...   
6816                                      0.303512  ...   
6817                                      0.303498  ...   
6818                                      0.313415  ...   

       Net Income to Total Assets   Total assets to GNP price  \
6814                     0.799927                    0.000466   
6815                     0.799748                    0.001959   
6816                     0.797778                    0.002840   
6817                     0.811808                    0.002837   
6818                     0.815956                    0.000707   

       No-credit Interval   Gross Profit to Sales  \
6814             0.623620                0.604455   
6815             0.623931                0.598306   
6816             0.624156                0.610441   
6817             0.623957                0.607846   
6818             0.626680                0.627408   

       Net Income to Stockholder's Equity   Liability to Equity  \
6814                             0.840359              0.279606   
6815                             0.840306              0.278132   
6816                             0.840138              0.275789   
6817                             0.841084              0.277547   
6818                             0.841019              0.275114   

       Degree of Financial Leverage (DFL)  \
6814                             0.027064   
6815                             0.027009   
6816                             0.026791   
6817                             0.026822   
6818                             0.026793   

       Interest Coverage Ratio (Interest expense to EBIT)   Net Income Flag  \
6814                                           0.566193                   1   
6815                       

In [16]:
# Summary statistics
bankruptcy_df.describe()

Bankrupt   ROA(C) before interest and depreciation before interest  \
count  6819.000000                                        6819.000000          
mean      0.032263                                           0.505180          
std       0.176710                                           0.060686          
min       0.000000                                           0.000000          
25%       0.000000                                           0.476527          
50%       0.000000                                           0.502706          
75%       0.000000                                           0.535563          
max       1.000000                                           1.000000          

        ROA(A) before interest and % after tax  \
count                              6819.000000   
mean                                  0.558625   
std                                   0.065620   
min                                   0.000000   
25%                                   0.535543   
50%                                   0.559802   
75%                                   0.589157   
max                                   1.000000   

        ROA(B) before interest and depreciation after tax  \
count                                        6819.000000    
mean                                            0.553589    
std                                             0.061595    
min                                             0.000000    
25%                                             0.527277    
50%                                             0.552278    
75%                                             0.584105    
max                                             1.000000    

        Operating Gross Margin   Realized Sales Gross Margin  \
count              6819.000000                   6819.000000   
mean                  0.607948                      0.607929   
std                   0.016934                      0.016916   
min                   0.000000                      0.000000   
25%                   0.600445                      0.600434   
50%                   0.605997                      0.605976   
75%                   0.613914                      0.613842   
max                   1.000000                      1.000000   

        Operating Profit Rate   Pre-tax net Interest Rate  \
count             6819.000000                 6819.000000   
mean                 0.998755                    0.797190   
std                  0.013010                    0.012869   
min                  0.000000                    0.000000   
25%                  0.998969                    0.797386   
50%                  0.999022                    0.797464   
75%                  0.999095                    0.797579   
max                  1.000000                    1.000000   

        After-tax net Interest Rate  \
count                   6819.000000   
mean                       0.809084   
std                        0.013601   
min                        0.000000   
25%                        0.809312   
50%                        0.809375   
75%                        0.809469   
max                        1.000000   

        Non-industry income and expenditure/revenue  ...  \
count                                   6819.000000  ...   
mean                                       0.303623  ...   
std                                        0.011163  ...   
min                                        0.000000  ...   
25%                                        0.303466  ...   
50%                                        0.303525  ...   
75%                                        0.303585  ...   
max                                        1.000000  ...   

        Net Income to Total Assets   Total assets to GNP price  \
count                  6819.000000                6.819000e+03   
mean                      0.807760                1.862942e+07   
std                       0.040332                3.764501e+08   
min 

In [17]:
# SweetViz
my_report = sv.analyze(bankruptcy_df)

                                             |          | [  0%]   00:00 -> (? left)

In [18]:
my_report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [19]:
bankruptcy_df["Bankrupt"].value_counts()

0    6599
1     220
Name: Bankrupt, dtype: int64

According to the Sweetviz report, the dataset, particularly the Bankrupt varible is imbalanced, with 6,599 observations for class 0 (97%) and 220 observations for class 1 (3%).

### Undersampling the dataset

In [20]:
# Separate the dataset into two datasets
dataset_1 = bankruptcy_df[bankruptcy_df["Bankrupt"] == 0]  # Class 0 (majority class)
dataset_2 = bankruptcy_df[bankruptcy_df["Bankrupt"] == 1]  # Class 1 (minority class)

In [21]:
# Check the number of observations in dataset_1
len(dataset_1)

6599

In [22]:
# Check the number of observations in dataset_2
len(dataset_2)

220

In [23]:
# Undersample the majority class
undersampled_dataset_1 = resample(dataset_1, replace=False, n_samples=220, random_state=42)

In [24]:
# Concatenate the new undersampled majority dataset to the minority dataset
undersampled_df = pd.concat([undersampled_dataset_1, dataset_2])

In [25]:
# Save dataset
undersampled_df.to_csv('/content/undersampled_df.csv', index=True)

In [26]:
undersampled_df

Bankrupt   ROA(C) before interest and depreciation before interest  \
2236         0                                           0.471945          
5538         0                                           0.507093          
4593         0                                           0.503924          
6315         0                                           0.451275          
4205         0                                           0.533418          
...        ...                                                ...          
6591         1                                           0.418515          
6640         1                                           0.196802          
6641         1                                           0.337640          
6642         1                                           0.340028          
6728         1                                           0.492176          

       ROA(A) before interest and % after tax  \
2236                                 0.540667   
5538                                 0.554187   
4593                                 0.550425   
6315                                 0.498528   
4205                                 0.613607   
...                                       ...   
6591                                 0.433984   
6640                                 0.211023   
6641                                 0.254307   
6642                                 0.344636   
6728                                 0.544320   

       ROA(B) before interest and depreciation after tax  \
2236                                           0.523636    
5538                                           0.558702    
4593                                           0.556936    
6315                                           0.503346    
4205                                           0.596445    
...                                                 ...    
6591                                           0.461427    
6640                                           0.221425    
6641                                           0.378446    
6642                                           0.380213    
6728                                           0.533326    

       Operating Gross Margin   Realized Sales Gross Margin  \
2236                 0.607518                      0.607518   
5538                 0.611972                      0.611986   
4593                 0.605788                      0.605788   
6315                 0.598438                      0.598438   
4205                 0.613939                      0.612743   
...                       ...                           ...   
6591                 0.612750                      0.612750   
6640                 0.598056                      0.598056   
6641                 0.590842                      0.590842   
6642                 0.581466                      0.581466   
6728                 0.618105                      0.618105   

       Operating Profit Rate   Pre-tax net Interest Rate  \
2236                0.999034                    0.797471   
5538                0.999132                    0.797542   
4593                0.999048                    0.797459   
6315                0.998972                    0.797280   
4205                0.999142                    0.797527   
...                      ...                         ...   
6591                0.998864                    0.796902   
6640                0.998933                    0.796144   
6641                0.998869                    0.796943   
6642                0.998372                    0.796292   
6728                0.999083                    0.797456   

       After-tax net Interest Rate  \
2236                      0.809381   
5538                      0.809430   
4593                      0.809370   
6315                      0.809214   
4205                      0.809452   
...                            ...   
6591                      0.808857   
6

In [27]:
# Check the number of observations for Class 0 of the new oversampled dataset
len(undersampled_df[undersampled_df["Bankrupt"]==0])

220

### Spliting the datasets into Training and Testing Sets

**Original dataset**

In [56]:
# Separate dataset into Y (dependent) and X (independent) variables
y = bankruptcy_df['Bankrupt']
X = bankruptcy_df.drop('Bankrupt', axis=1)

In [57]:
# Use the train_test_split function to split the sale data into training and testing set (80% vs 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Undersampled dataset**

In [58]:
# Separate dataset into Y (dependent) and X (independent) variables
y_undersampled = undersampled_df['Bankrupt']
X_undersampled = undersampled_df.drop('Bankrupt', axis=1)

In [59]:
# Use the train_test_split function to split the loan data into training and testing set (80% vs 20%)
X_train_undersampled, X_test_undersampled, y_train_undersampled, y_test_undersampled = train_test_split(X_undersampled, y_undersampled, test_size=0.2, random_state=5)

## Part I

### Random Forest

**1. Fit the model on the training set using the default parameters and report your findings.**

In [60]:
# Instantiate the model
clf_rf = RandomForestClassifier()

In [61]:
# Fit the model
%%time
clf_rf.fit(X_train, y_train)

CPU times: user 3.65 s, sys: 7.78 ms, total: 3.66 s
Wall time: 3.69 s


RandomForestClassifier()

In [62]:
# Make predictions on test set
y_pred = clf_rf.predict(X_test)

In [63]:
# Evaluation metrics (I avoided using accuracy because the original dataset is imbalanced)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred)

In [64]:
precision, recall, f_score

(0.8888888888888888, 0.1568627450980392, 0.26666666666666666)

* A precision of 0.8889 indicates that when the classifier predicts a company as bankrupt, it is correct about 88.89% of the time.
* A recall of 0.1569 indicates that the classifier is capturing only about 15.69% of the actual bankrupt companies.
* The model has high precision, however, the recall is relatively low. This means that when the model predicts bankruptcy, it is correct a large percentage of the time, but it misses a substantial number of actual bankrupt cases.
* An F-score of 0.2667 indicates the trade-off between precision recall. The model is not achieving a satisfactory balance between precision and recall.

In [65]:
# Display the decision path
clf_rf.decision_path(X_train)

(<5455x21254 sparse matrix of type '<class 'numpy.int64'>'
 	with 5338611 stored elements in Compressed Sparse Row format>,
 array([    0,   219,   448,   687,   900,  1099,  1318,  1519,  1730,
         1955,  2168,  2393,  2594,  2805,  2984,  3223,  3422,  3651,
         3866,  4081,  4300,  4497,  4692,  4911,  5118,  5311,  5540,
         5763,  5948,  6187,  6402,  6615,  6830,  7059,  7280,  7479,
         7682,  7885,  8106,  8317,  8534,  8745,  8934,  9141,  9362,
         9571,  9756,  9951, 10170, 10381, 10586, 10797, 10998, 11233,
        11446, 11685, 11902, 12111, 12312, 12543, 12748, 12961, 13172,
        13339, 13548, 13745, 13930, 14139, 14382, 14623, 14836, 15069,
        15276, 15499, 15704, 15939, 16160, 16373, 16580, 16803, 17010,
        17231, 17434, 17651, 17850, 18059, 18288, 18503, 18718, 18925,
        19130, 19337, 19552, 19777, 19992, 20203, 20406, 20641, 20854,
        21055, 21254]))

In [66]:
# Pairing feature names with their importances
feature_importance_pairs = list(zip(X_train.columns, clf_rf.feature_importances_))

# Displaying the feature importances alongside their names
for feature, importance in feature_importance_pairs:
    print(f"{feature}: {importance}")

 ROA(C) before interest and depreciation before interest: 0.013561483122392685
 ROA(A) before interest and % after tax: 0.011312083660149757
 ROA(B) before interest and depreciation after tax: 0.010715766714336498
 Operating Gross Margin: 0.00619813084229941
 Realized Sales Gross Margin: 0.006206781812297031
 Operating Profit Rate: 0.005616937231320381
 Pre-tax net Interest Rate: 0.010460711423863758
 After-tax net Interest Rate: 0.009620457375436214
 Non-industry income and expenditure/revenue: 0.018033978155599078
 Continuous interest rate (after tax): 0.012508061435248236
 Operating Expense Rate: 0.009768870398829007
 Research and development expense rate: 0.006508726765356376
 Cash flow rate: 0.006920925901944419
 Interest-bearing debt interest rate: 0.01661466815668654
 Tax rate (A): 0.0024438870503382276
 Net Value Per Share (B): 0.014323912128276276
 Net Value Per Share (A): 0.013637673695433568
 Net Value Per Share (C): 0.020777944699338944
 Persistent EPS in the Last Four Seas

**2. Repeat 1 but change the parameters to different non-default parameters. Evaluate this model on your choice of metrics. Which model do you prefer?**

In [67]:
# Random Forest Classifier with Non-default Parameters
clf_rf_modified = RandomForestClassifier(criterion='entropy')

In [68]:
# Fit the model
%%time
clf_rf_modified.fit(X_train, y_train)

CPU times: user 3.54 s, sys: 6.38 ms, total: 3.55 s
Wall time: 5.89 s


RandomForestClassifier(criterion='entropy')

In [69]:
# Make predictions on test set
y_pred_modified = clf_rf_modified.predict(X_test)

In [70]:
# Calculate metrics
precision_modified = precision_score(y_test, y_pred_modified)
recall_modified = recall_score(y_test, y_pred_modified)
f_score_modified = f1_score(y_test, y_pred_modified)

In [71]:
precision_modified, recall_modified, f_score_modified

(0.7272727272727273, 0.1568627450980392, 0.25806451612903225)

* A precision of 0.7273 indicates that when the classifier predicts a company as bankrupt, it is correct about 72.73% of the time.
* A recall of 15.69 indicates that the classifier is capturing only about 15.69% of the actual bankrupt companies.
* The model's precision is relatively high, while the recall is low.
* An F-score of 0.2581 indicates the trade-off between precision recall. The model is not achieving a satisfactory balance between precision and recall.
I prefer the Random Forest Classifier with Default parameters since it has better (higher) metrics compared to the classifier with non-default parameters.

**3. Which model takes longer to fit? 1. Or 2.? Report the CPU time.**

The Random Forest Classifier with Default parameters takes longer to fit.
* The Random Forest Classifier with Default parameters: user 3.65 s, sys: 7.78 ms, total: 3.66 s
* The Random Forest Classifier with Non-default parameters: user 3.54 s, sys: 6.38 ms, total: 3.55 s

**4. Based on your findings above which parameter combinations give you the best results for classification? Would you prefer “Gini” or “entropy” as your splitting metric?**

In order to identify which parameter combinations yield the best results for classification, I believe we need to conduct a hyperparameter tuning process through cross-validation, involving fine-tuning hyperparameters, such as n_estimators, to optimize the model's performance on the given dataset.

Based on my findings above, I would prefer the "Gini" metric (or the Default Random Forest Classifier) as my splitting criterion since the Default Random Forest Classifier has a higher metrics compared to the Random Forest Classifier with non-default parameters' metrics.

**5. Choose one of the models from number 1. Or number 2, display, and discuss the decision rules. Do the rules make sense for classification?**

In [72]:
# Display the decision rules
for i, clf in enumerate(clf_rf.estimators_):
    # Display decision rules as text
    text_representation = export_text(clf, feature_names=X_train.columns.tolist())
    print(f"Decision Rules:\n{text_representation}")

Streaming output truncated to the last 5000 lines.
|   |   |---  Continuous Net Profit Growth Rate >  0.22
|   |   |   |---  Allocation rate per person <= 0.25
|   |   |   |   |---  Net Value Growth Rate <= 0.00
|   |   |   |   |   |---  Total assets to GNP price <= 0.00
|   |   |   |   |   |   |--- class: 0.0
|   |   |   |   |   |---  Total assets to GNP price >  0.00
|   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |---  Net Value Growth Rate >  0.00
|   |   |   |   |   |---  Current Ratio <= 0.01
|   |   |   |   |   |   |---  Current Liability to Current Assets <= 0.04
|   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |---  Current Liability to Current Assets >  0.04
|   |   |   |   |   |   |   |---  Average Collection Days <= 0.01
|   |   |   |   |   |   |   |   |---  Interest Expense Ratio <= 0.63
|   |   |   |   |   |   |   |   |   |---  Operating profit/Paid-in capital <= 0.09
|   |   |   |   |   |   |   |   |   |   |---  Interest-bearing debt interest r

The decision rules display a hierarchical structure for classification, providing a clear path for determining the class of a given entity based on specific financial parameters (financial ratios).

The rules use various financial ratios. The decision tree makes logical sense for classification, as it systematically navigates through different financial metrics to arrive at a final classification of either class 0.0 or class 1.0.

Based on the Random Forest Classifier, if the Continuous Net Profit Growth Rate is larger than 0.22, the algorithm proceeds to examine the Allocation Rate per person. If this rate is less than or equal to 0.25, it checks the Net Value Growth Rate, verifying it is below 0.00. The algorithm continues by evaluating the Total Assets to GNP Price. If it does not exceed 0.00, the final decision is made, and the instance is classified as belonging to Class 0; else, if the Total Assets to GNP Price is larger than 0.00, the instance is classified as belonging to Class 1.

Therefore, the rules make sense for classification

**6. Which features are the most important for classification?**

In [73]:
# Get and pair feature importances
feature_importances = dict(zip(X.columns, clf_rf.feature_importances_))

# Top 5 most important features
top_features = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)[:5]
top_features

[(" Net Income to Stockholder's Equity", 0.028528101684305578),
 (' Net Value Growth Rate', 0.02712142741165896),
 (' Persistent EPS in the Last Four Seasons', 0.023315610984259494),
 (' Net Value Per Share (C)', 0.020777944699338944),
 (' Per Share Net profit before tax (Yuan ¥)', 0.018961126241586357)]

**7. On average, what do you think is the tradeoff between model fitting time and model performance?**

According to the above outputs, I think the tradeoff between model fitting time and model performance is that the better the model performs, the longer it takes to fit the model: The Default Random Forest Classifier outperforms the Random Forest Classifier with non-default parameters because it has higher metrics. Also, the Default Random Forest Classifier has a longer CPU time compared to the Random Forest Classifier with non-default parameters.

### Support Vector Machines

**1. Fit the model on the training set using the default parameters and report your findings.**

In [74]:
# Instantiate the model
clf = SVC()

In [75]:
# Fit the model
%%time
clf.fit(X_train, y_train)

CPU times: user 688 ms, sys: 4.39 ms, total: 692 ms
Wall time: 694 ms


SVC()

In [76]:
# Make predictions on test set
y_pred_clf = clf.predict(X_test)

In [77]:
# Calculate metrics
accuracy_clf = accuracy_score(y_test, y_pred_clf)
precision_clf_zero_division = precision_score(y_test, y_pred_clf, zero_division=1)
precision_clf = precision_score(y_test, y_pred_clf)

In [78]:
accuracy_clf, precision_clf_zero_division, precision_clf

(0.9626099706744868, 1.0, 0.0)

In [79]:
# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_clf).ravel()
tn, fp, fn, tp

(1313, 0, 51, 0)

* True Negatives (TN) is 1313, indicating the correct prediction of instances belonging to Class 0 (negative).
* False Positives (FP) is 0, implying no instances were mistakenly classified as Class 1 (positive) when they actually pertain to Class 0.
* False Negatives (FN) is 51, indicating the incorrect prediction of Class 0 when they should have been assigned to Class 1.
* True Positives (TP) is 0, indicating the model's inability to successfully identify any instances of Class 1.
The significant number of True Negatives contributes to a high accuracy, but the model encounters challenges in accurately classifying instances from Class 1, leading to undefined precision, recall, and F1 score for Class 1. This issue could stem from a strong imbalance in the distribution of the classes.

**2. Change the kernel from “rbf” to “linear” and repeat the model fitting procedure in 1 above.**

In [80]:
# Instantiate the model
clf_modified = SVC(kernel='linear')

In [53]:
# Fit the model
# clf_modified.fit(X_train, y_train)
# The linear SVM did not converge.

**3. Evaluate both models using any metrics of choice.**

In [81]:
# Calculate metrics
accuracy_clf = accuracy_score(y_test, y_pred_clf)
precision_clf_zero_division = precision_score(y_test, y_pred_clf, zero_division=1)
precision_clf = precision_score(y_test, y_pred_clf)

In [82]:
accuracy_clf, precision_clf_zero_division, precision_clf

(0.9626099706744868, 1.0, 0.0)

* An accuracy of 0.9626 indicates that the model is correct about 96.26% of the time. However, because this dataset is significantly imbalanced, accuracy may not be a good evaluation metric.
* While the significant number of True Negatives contributes to high accuracy, the model encounters challenges in accurately classifying instances from Class 1, leading to undefined precision, recall, and F1 score for Class 1. This issue could stem from a strong imbalance in the distribution of the classes.
* The precision value with zero division is 1.0, implying that there were no false positives or instances predicted as positive by the model.

**4. How long does it take to fit the SVM model in 1. above? How about 2. above? Report only the CPU times.**

* The Default Support Vector Machines (Model 1): user 688 ms, sys: 4.39 ms, total: 692 ms
* The linear SVM (Model 2) did not converge.

## Part II

**1. Fit the Random Forest and the SVM on the undersampled data and compare the fitting times for both models? Which model takes longer to fit?**

**Random Forest**

In [83]:
# Instantiate the model
clf_rf_undersampled = RandomForestClassifier()

In [84]:
# Fit the model
%%time
clf_rf_undersampled.fit(X_train_undersampled, y_train_undersampled)

CPU times: user 454 ms, sys: 4.38 ms, total: 459 ms
Wall time: 478 ms


RandomForestClassifier()

**SVM**

In [85]:
# Instantiate the model
clf_undersampled = SVC()

In [86]:
# Fit the model
%%time
clf_undersampled.fit(X_train_undersampled, y_train_undersampled)

CPU times: user 23.1 ms, sys: 585 µs, total: 23.7 ms
Wall time: 36.4 ms


SVC()

The Default Random Forest Classifier takes longer to fit.

* Default Random Forest Classifier: user 454 ms, sys: 4.38 ms, total: 459 ms

* Default Support Vector Machines: user 23.1 ms, sys: 585 µs, total: 23.7 ms

**2. Compare the performance of both models on any metrics of choice.**

In [87]:
# Random Forest
y_pred_rf_undersampled = clf_rf_undersampled.predict(X_test_undersampled)

# SVM
y_pred_clf_undersampled = clf_undersampled.predict(X_test_undersampled)

# Evaluate models based on accuracy
accuracy_rf_undersampled = accuracy_score(y_test_undersampled, y_pred_rf_undersampled)
accuracy_svm_undersampled = accuracy_score(y_test_undersampled, y_pred_clf_undersampled)

In [88]:
# Accuracy of Randome Forest
accuracy_rf_undersampled

0.7954545454545454

In [89]:
# Accuracy of SVM
accuracy_svm_undersampled

0.625

Random Forest performs better based on accuracy.